# E-commerce Business Analytics Dashboard

## Executive Summary

This notebook provides comprehensive analysis of e-commerce business performance, focusing on revenue trends, product performance, customer experience, and geographic distribution. The analysis is designed to be configurable for different time periods and provides actionable business insights.

### Key Business Objectives
- Monitor revenue growth and identify trends
- Analyze product category performance and market opportunities
- Evaluate customer experience and operational efficiency
- Assess geographic market penetration
- Track key performance indicators (KPIs) for business health

---

## Table of Contents

1. [Configuration & Setup](#1-configuration--setup)
2. [Data Dictionary](#2-data-dictionary)
3. [Data Loading & Preparation](#3-data-loading--preparation)
4. [Revenue Analysis](#4-revenue-analysis)
5. [Product Performance Analysis](#5-product-performance-analysis)
6. [Geographic Analysis](#6-geographic-analysis)
7. [Customer Experience Analysis](#7-customer-experience-analysis)
8. [Business Health Dashboard](#8-business-health-dashboard)
9. [Key Insights & Recommendations](#9-key-insights--recommendations)

---

## 1. Configuration & Setup

Configure the analysis parameters including date ranges, comparison periods, and visualization settings.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Import custom modules
from data_loader import create_analysis_dataset, get_data_summary
from business_metrics import (
    calculate_revenue_metrics, 
    calculate_monthly_trends,
    analyze_product_performance,
    analyze_geographic_performance,
    analyze_customer_experience,
    calculate_business_health_score,
    generate_executive_summary
)

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configure plotly for business-friendly colors
BUSINESS_COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72', 
    'success': '#27AE60',
    'warning': '#F39C12',
    'danger': '#E74C3C',
    'neutral': '#7F8C8D'
}

print("Environment configured successfully!")

In [ ]:
# ANALYSIS CONFIGURATION
# Modify these parameters to analyze different time periods

# Primary analysis period
ANALYSIS_YEAR = 2023
ANALYSIS_MONTH = None  # Set to specific month (1-12) or None for full year

# Comparison period (for growth calculations)
COMPARISON_YEAR = 2022
COMPARISON_MONTH = None  # Set to specific month (1-12) or None for full year

# Alternative: Use date ranges instead of year/month
USE_DATE_RANGE = False
START_DATE = '2023-01-01'  # Format: 'YYYY-MM-DD'
END_DATE = '2023-12-31'    # Format: 'YYYY-MM-DD'
COMPARISON_START_DATE = '2022-01-01'
COMPARISON_END_DATE = '2022-12-31'

# Data source configuration
DATA_PATH = 'ecommerce_data/'

# Visualization settings
FIGURE_SIZE = (12, 6)
TOP_N_CATEGORIES = 10
TOP_N_STATES = 10

print(f"Analysis configured for: {ANALYSIS_YEAR}" + (f"-{ANALYSIS_MONTH:02d}" if ANALYSIS_MONTH else ""))
print(f"Comparison period: {COMPARISON_YEAR}" + (f"-{COMPARISON_MONTH:02d}" if COMPARISON_MONTH else ""))

## 2. Data Dictionary

Understanding the key data elements and business terminology used in this analysis.

### Core Business Metrics

| Metric | Definition | Business Impact |
|--------|------------|----------------|
| **Total Revenue** | Sum of all delivered order values (price + freight) | Primary indicator of business performance |
| **Average Order Value (AOV)** | Total revenue divided by number of orders | Measures customer spending behavior |
| **Order Volume** | Total number of unique orders | Indicates market demand and reach |
| **Revenue Growth Rate** | Percentage change in revenue vs comparison period | Shows business trajectory |
| **Customer Review Score** | Average rating on 1-5 scale | Measures customer satisfaction |
| **Delivery Performance** | Average days from purchase to delivery | Operational efficiency indicator |

### Data Sources

| Dataset | Key Columns | Purpose |
|---------|-------------|----------|
| **orders_dataset.csv** | order_id, customer_id, order_status, timestamps | Order lifecycle and customer mapping |
| **order_items_dataset.csv** | order_id, product_id, price, freight_value | Revenue and product details |
| **products_dataset.csv** | product_id, product_category_name | Product categorization |
| **customers_dataset.csv** | customer_id, customer_state, customer_city | Geographic analysis |
| **order_reviews_dataset.csv** | order_id, review_score | Customer satisfaction metrics |

### Business Rules

- **Analysis Scope**: Only delivered orders are included in revenue calculations
- **Revenue Calculation**: Total Revenue = Item Price + Freight Value
- **Time Attribution**: Orders are attributed to the purchase timestamp date
- **Geographic Attribution**: Based on customer billing address
- **Review Scoring**: 1 = Very Dissatisfied, 5 = Very Satisfied


## 3. Data Loading & Preparation

Loading and preparing the e-commerce dataset with proper data quality checks and transformations.

In [ ]:
# Load primary analysis dataset
if USE_DATE_RANGE:
    primary_data = create_analysis_dataset(
        data_path=DATA_PATH,
        start_date=START_DATE,
        end_date=END_DATE
    )
    period_label = f"{START_DATE} to {END_DATE}"
else:
    primary_data = create_analysis_dataset(
        data_path=DATA_PATH,
        year=ANALYSIS_YEAR,
        month=ANALYSIS_MONTH
    )
    period_label = f"{ANALYSIS_YEAR}" + (f"-{ANALYSIS_MONTH:02d}" if ANALYSIS_MONTH else "")

# Load comparison dataset
if USE_DATE_RANGE:
    comparison_data = create_analysis_dataset(
        data_path=DATA_PATH,
        start_date=COMPARISON_START_DATE,
        end_date=COMPARISON_END_DATE
    )
    comparison_label = f"{COMPARISON_START_DATE} to {COMPARISON_END_DATE}"
else:
    comparison_data = create_analysis_dataset(
        data_path=DATA_PATH,
        year=COMPARISON_YEAR,
        month=COMPARISON_MONTH
    )
    comparison_label = f"{COMPARISON_YEAR}" + (f"-{COMPARISON_MONTH:02d}" if COMPARISON_MONTH else "")

In [ ]:
# Data quality summary
print("PRIMARY ANALYSIS DATASET")
print("=" * 40)
get_data_summary(primary_data)

print("\n\nCOMPARISON DATASET")
print("=" * 40)
get_data_summary(comparison_data)

In [ ]:
# Display sample of enriched dataset
print("Sample of analysis-ready dataset:")
display(primary_data[[
    'order_id', 'product_category_name', 'customer_state', 
    'price', 'total_revenue', 'review_score', 'delivery_days'
]].head(10))

## 4. Revenue Analysis

Comprehensive analysis of revenue performance, growth trends, and key drivers.

In [ ]:
# Calculate core revenue metrics
revenue_metrics = calculate_revenue_metrics(primary_data, comparison_data)
monthly_trends = calculate_monthly_trends(primary_data)

# Display key metrics
print(f"REVENUE PERFORMANCE - {period_label.upper()}")
print("=" * 50)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:.2f}")
print(f"Total Items Sold: {revenue_metrics['total_items_sold']:,}")

if 'revenue_growth_rate' in revenue_metrics:
    print("\nGROWTH METRICS vs COMPARISON PERIOD")
    print("-" * 35)
    print(f"Revenue Growth: {revenue_metrics['revenue_growth_rate']:.2%}")
    print(f"Order Growth: {revenue_metrics['order_growth_rate']:.2%}")
    print(f"AOV Growth: {revenue_metrics['aov_growth_rate']:.2%}")

In [ ]:
# Monthly revenue trend visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'Monthly Revenue Trend - {period_label}',
        f'Monthly Order Volume - {period_label}',
        'Month-over-Month Revenue Growth',
        'Average Order Value Trend'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Monthly revenue
fig.add_trace(
    go.Scatter(
        x=monthly_trends['month'],
        y=monthly_trends['revenue'],
        mode='lines+markers',
        name='Revenue',
        line=dict(color=BUSINESS_COLORS['primary'], width=3),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# Monthly orders
fig.add_trace(
    go.Scatter(
        x=monthly_trends['month'],
        y=monthly_trends['orders'],
        mode='lines+markers',
        name='Orders',
        line=dict(color=BUSINESS_COLORS['secondary'], width=3),
        marker=dict(size=8)
    ),
    row=1, col=2
)

# MoM growth
colors = [BUSINESS_COLORS['success'] if x > 0 else BUSINESS_COLORS['danger'] 
          for x in monthly_trends['revenue_mom_growth'].fillna(0)]

fig.add_trace(
    go.Bar(
        x=monthly_trends['month'],
        y=monthly_trends['revenue_mom_growth'] * 100,
        name='Growth %',
        marker_color=colors
    ),
    row=2, col=1
)

# AOV trend
fig.add_trace(
    go.Scatter(
        x=monthly_trends['month'],
        y=monthly_trends['avg_order_value'],
        mode='lines+markers',
        name='AOV',
        line=dict(color=BUSINESS_COLORS['warning'], width=3),
        marker=dict(size=8)
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    showlegend=False,
    title_text=f"Revenue Performance Dashboard - {period_label}",
    title_x=0.5
)

# Update y-axis labels
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Number of Orders", row=1, col=2)
fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=1)
fig.update_yaxes(title_text="Average Order Value ($)", row=2, col=2)

# Update x-axis labels
fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_xaxes(title_text="Month", row=2, col=2)

fig.show()

### Revenue Analysis Insights

Key observations from the revenue analysis:

- **Performance vs Previous Period**: The analysis shows revenue and order trends compared to the baseline period
- **Seasonal Patterns**: Monthly trends reveal seasonal variations in business performance
- **Growth Momentum**: Month-over-month growth rates indicate business acceleration or deceleration
- **Order Value Trends**: AOV changes suggest shifts in customer purchasing behavior

## 5. Product Performance Analysis

Analysis of product categories, market share, and performance drivers.

In [ ]:
# Calculate product performance metrics
product_metrics = analyze_product_performance(primary_data, top_n=TOP_N_CATEGORIES)

# Display top performing categories
print(f"TOP PRODUCT CATEGORIES - {period_label.upper()}")
print("=" * 50)
display(product_metrics['category_performance'][[
    'category', 'total_revenue', 'unique_orders', 'items_sold', 'avg_price'
]].head(10))

print("\nMARKET SHARE BY CATEGORY")
print("-" * 30)
market_share_display = product_metrics['market_share'].copy()
market_share_display['market_share_pct'] = (market_share_display['market_share'] * 100).round(2)
display(market_share_display[['category', 'market_share_pct']].head(10))

In [ ]:
# Product category performance visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'Revenue by Product Category - {period_label}',
        f'Market Share Distribution - {period_label}',
        'Average Price by Category',
        'Order Volume by Category'
    ),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Top categories by revenue
top_categories = product_metrics['category_performance'].head(8)

fig.add_trace(
    go.Bar(
        x=top_categories['category'],
        y=top_categories['total_revenue'],
        name='Revenue',
        marker_color=BUSINESS_COLORS['primary']
    ),
    row=1, col=1
)

# Market share pie chart
market_share_data = product_metrics['market_share'].head(6)
fig.add_trace(
    go.Pie(
        labels=market_share_data['category'],
        values=market_share_data['market_share'],
        name='Market Share'
    ),
    row=1, col=2
)

# Average price by category
fig.add_trace(
    go.Bar(
        x=top_categories['category'],
        y=top_categories['avg_price'],
        name='Avg Price',
        marker_color=BUSINESS_COLORS['warning']
    ),
    row=2, col=1
)

# Order volume by category
fig.add_trace(
    go.Bar(
        x=top_categories['category'],
        y=top_categories['unique_orders'],
        name='Orders',
        marker_color=BUSINESS_COLORS['secondary']
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=900,
    showlegend=False,
    title_text=f"Product Category Performance - {period_label}",
    title_x=0.5
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)

# Update y-axis labels
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Average Price ($)", row=2, col=1)
fig.update_yaxes(title_text="Number of Orders", row=2, col=2)

fig.show()

### Product Performance Insights

Key findings from product category analysis:

- **Revenue Leaders**: Top-performing categories drive the majority of business revenue
- **Market Concentration**: Market share distribution shows concentration levels across categories
- **Price Points**: Average pricing varies significantly across product categories
- **Volume vs Value**: Some categories drive high order volumes while others focus on higher-value transactions

## 6. Geographic Analysis

Geographic distribution of sales performance and market penetration analysis.

In [ ]:
# Calculate geographic performance metrics
geo_metrics = analyze_geographic_performance(primary_data, top_n=TOP_N_STATES)

# Display top performing states
print(f"TOP PERFORMING STATES - {period_label.upper()}")
print("=" * 50)
display(geo_metrics['state_performance'][[
    'state', 'total_revenue', 'total_orders', 'unique_customers', 'revenue_per_customer'
]].head(10))

print("\nTOP CUSTOMER CONCENTRATION STATES")
print("-" * 35)
display(geo_metrics['top_customer_states'].head(10))

In [ ]:
# Geographic performance visualization
state_data = geo_metrics['state_performance'].head(15)

# Create choropleth map
fig_map = px.choropleth(
    state_data,
    locations='state',
    color='total_revenue',
    locationmode='USA-states',
    scope='usa',
    title=f'Revenue by State - {period_label}',
    color_continuous_scale='Blues',
    labels={'total_revenue': 'Revenue ($)'}
)

fig_map.update_layout(height=600)
fig_map.show()

# State performance bar charts
fig_bars = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Top States by Revenue',
        'Top States by Order Volume',
        'Top States by Customer Count',
        'Revenue per Customer by State'
    )
)

top_10_states = state_data.head(10)

# Revenue by state
fig_bars.add_trace(
    go.Bar(
        x=top_10_states['state'],
        y=top_10_states['total_revenue'],
        name='Revenue',
        marker_color=BUSINESS_COLORS['primary']
    ),
    row=1, col=1
)

# Orders by state
fig_bars.add_trace(
    go.Bar(
        x=top_10_states['state'],
        y=top_10_states['total_orders'],
        name='Orders',
        marker_color=BUSINESS_COLORS['secondary']
    ),
    row=1, col=2
)

# Customers by state
fig_bars.add_trace(
    go.Bar(
        x=top_10_states['state'],
        y=top_10_states['unique_customers'],
        name='Customers',
        marker_color=BUSINESS_COLORS['success']
    ),
    row=2, col=1
)

# Revenue per customer
fig_bars.add_trace(
    go.Bar(
        x=top_10_states['state'],
        y=top_10_states['revenue_per_customer'],
        name='Revenue per Customer',
        marker_color=BUSINESS_COLORS['warning']
    ),
    row=2, col=2
)

fig_bars.update_layout(
    height=800,
    showlegend=False,
    title_text=f"Geographic Performance Analysis - {period_label}",
    title_x=0.5
)

# Update y-axis labels
fig_bars.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig_bars.update_yaxes(title_text="Number of Orders", row=1, col=2)
fig_bars.update_yaxes(title_text="Number of Customers", row=2, col=1)
fig_bars.update_yaxes(title_text="Revenue per Customer ($)", row=2, col=2)

fig_bars.show()

### Geographic Analysis Insights

Key geographic performance indicators:

- **Market Concentration**: Revenue concentration across different states reveals key markets
- **Customer Density**: States with high customer counts may indicate strong market penetration
- **Revenue Efficiency**: Revenue per customer varies by state, indicating market maturity or pricing differences
- **Growth Opportunities**: Lower-performing states may represent expansion opportunities

## 7. Customer Experience Analysis

Analysis of customer satisfaction, delivery performance, and service quality metrics.

In [ ]:
# Calculate customer experience metrics
cx_metrics = analyze_customer_experience(primary_data)

# Display customer experience summary
print(f"CUSTOMER EXPERIENCE METRICS - {period_label.upper()}")
print("=" * 55)

if 'avg_review_score' in cx_metrics:
    print(f"Average Review Score: {cx_metrics['avg_review_score']:.2f}/5.0")
    
if 'nps_score' in cx_metrics:
    print(f"Net Promoter Score: {cx_metrics['nps_score']:.1f}")

if 'avg_delivery_days' in cx_metrics:
    print(f"Average Delivery Time: {cx_metrics['avg_delivery_days']:.1f} days")
    print(f"Median Delivery Time: {cx_metrics['median_delivery_days']:.1f} days")

if 'review_score_distribution' in cx_metrics:
    print("\nREVIEW SCORE DISTRIBUTION")
    print("-" * 25)
    for score, pct in cx_metrics['review_score_distribution'].sort_index(ascending=False).items():
        print(f"{score} stars: {pct:.1%}")

if 'delivery_satisfaction' in cx_metrics:
    print("\nDELIVERY SPEED vs SATISFACTION")
    print("-" * 30)
    display(cx_metrics['delivery_satisfaction'])

In [ ]:
# Customer experience visualizations
fig_cx = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'Review Score Distribution - {period_label}',
        f'Delivery Performance - {period_label}',
        'Delivery Speed vs Customer Satisfaction',
        'Customer Satisfaction Trends'
    ),
    specs=[[{"type": "bar"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Review score distribution
if 'review_score_distribution' in cx_metrics:
    review_dist = cx_metrics['review_score_distribution'].sort_index()
    colors_review = [BUSINESS_COLORS['danger'] if x <= 2 else 
                    BUSINESS_COLORS['warning'] if x == 3 else 
                    BUSINESS_COLORS['success'] for x in review_dist.index]
    
    fig_cx.add_trace(
        go.Bar(
            x=review_dist.index,
            y=review_dist.values * 100,
            name='Review Distribution',
            marker_color=colors_review
        ),
        row=1, col=1
    )

# Delivery days histogram
delivery_data = primary_data.dropna(subset=['delivery_days'])
if len(delivery_data) > 0:
    fig_cx.add_trace(
        go.Histogram(
            x=delivery_data['delivery_days'],
            name='Delivery Days',
            marker_color=BUSINESS_COLORS['primary'],
            nbinsx=20
        ),
        row=1, col=2
    )

# Delivery speed vs satisfaction
if 'delivery_satisfaction' in cx_metrics and len(cx_metrics['delivery_satisfaction']) > 0:
    delivery_sat = cx_metrics['delivery_satisfaction']
    fig_cx.add_trace(
        go.Scatter(
            x=delivery_sat['delivery_category'],
            y=delivery_sat['review_score'],
            mode='markers+lines',
            name='Satisfaction vs Speed',
            marker=dict(size=delivery_sat['order_id']/10, color=BUSINESS_COLORS['secondary']),
            line=dict(color=BUSINESS_COLORS['secondary'])
        ),
        row=2, col=1
    )

# Customer satisfaction by delivery category
if 'delivery_category' in primary_data.columns:
    satisfaction_by_delivery = primary_data.groupby('delivery_category')['review_score'].mean().dropna()
    if len(satisfaction_by_delivery) > 0:
        fig_cx.add_trace(
            go.Bar(
                x=satisfaction_by_delivery.index,
                y=satisfaction_by_delivery.values,
                name='Avg Rating by Delivery Speed',
                marker_color=BUSINESS_COLORS['success']
            ),
            row=2, col=2
        )

fig_cx.update_layout(
    height=800,
    showlegend=False,
    title_text=f"Customer Experience Analysis - {period_label}",
    title_x=0.5
)

# Update y-axis labels
fig_cx.update_yaxes(title_text="Percentage (%)", row=1, col=1)
fig_cx.update_yaxes(title_text="Number of Orders", row=1, col=2)
fig_cx.update_yaxes(title_text="Average Review Score", row=2, col=1)
fig_cx.update_yaxes(title_text="Average Review Score", row=2, col=2)

# Update x-axis labels
fig_cx.update_xaxes(title_text="Review Score (1-5)", row=1, col=1)
fig_cx.update_xaxes(title_text="Delivery Days", row=1, col=2)
fig_cx.update_xaxes(title_text="Delivery Speed Category", row=2, col=1)
fig_cx.update_xaxes(title_text="Delivery Speed Category", row=2, col=2)

fig_cx.show()

### Customer Experience Insights

Key customer experience findings:

- **Overall Satisfaction**: Average review scores indicate general customer satisfaction levels
- **Delivery Performance**: Distribution of delivery times shows operational efficiency
- **Service Quality Impact**: Correlation between delivery speed and customer satisfaction
- **Improvement Opportunities**: Areas where customer experience can be enhanced

## 8. Business Health Dashboard

Overall business health assessment combining revenue, customer experience, and operational metrics.

In [ ]:
# Calculate business health scores
health_metrics = calculate_business_health_score(primary_data, comparison_data)
executive_summary = generate_executive_summary(primary_data, comparison_data, period_label)

# Display business health dashboard
print(f"BUSINESS HEALTH DASHBOARD - {period_label.upper()}")
print("=" * 60)
print(f"Overall Health Score: {health_metrics['overall_health_score']:.1f}/100")

print("\nCOMPONENT SCORES")
print("-" * 20)
print(f"Revenue Performance: {health_metrics['revenue_score']:.1f}/100")
print(f"Customer Experience: {health_metrics['customer_experience_score']:.1f}/100")
print(f"Operational Efficiency: {health_metrics['operational_score']:.1f}/100")
print(f"Order Fulfillment: {health_metrics['fulfillment_score']:.1f}/100")

print("\nEXECUTIVE SUMMARY")
print("-" * 20)
print(f"Total Revenue: ${executive_summary['total_revenue']:,.2f}")
print(f"Total Orders: {executive_summary['total_orders']:,}")
print(f"Average Order Value: ${executive_summary['average_order_value']:.2f}")

if 'revenue_growth_rate' in executive_summary:
    print(f"Revenue Growth: {executive_summary['revenue_growth_rate']:.2%}")

if 'avg_review_score' in executive_summary:
    print(f"Customer Satisfaction: {executive_summary['avg_review_score']:.2f}/5.0")

print(f"\nTop Product Categories: {', '.join(executive_summary['top_categories'])}")
print(f"Top Geographic Markets: {', '.join(executive_summary['top_states'])}")

In [ ]:
# Business health visualization
fig_health = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Business Health Score Breakdown',
        'Key Performance Indicators',
        'Revenue vs Customer Satisfaction',
        'Business Health Gauge'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "indicator"}]]
)

# Health score components
health_components = {
    'Revenue': health_metrics['revenue_score'],
    'Customer Experience': health_metrics['customer_experience_score'],
    'Operations': health_metrics['operational_score'],
    'Fulfillment': health_metrics['fulfillment_score']
}

colors_health = [BUSINESS_COLORS['success'] if score >= 80 else 
                BUSINESS_COLORS['warning'] if score >= 60 else 
                BUSINESS_COLORS['danger'] for score in health_components.values()]

fig_health.add_trace(
    go.Bar(
        x=list(health_components.keys()),
        y=list(health_components.values()),
        name='Health Scores',
        marker_color=colors_health
    ),
    row=1, col=1
)

# Key metrics comparison (if comparison data available)
if comparison_data is not None:
    metrics_comparison = {
        'Revenue': [executive_summary['total_revenue'], calculate_revenue_metrics(comparison_data)['total_revenue']],
        'Orders': [executive_summary['total_orders'], calculate_revenue_metrics(comparison_data)['total_orders']],
        'AOV': [executive_summary['average_order_value'], calculate_revenue_metrics(comparison_data)['average_order_value']]
    }
    
    for i, (metric, values) in enumerate(metrics_comparison.items()):
        fig_health.add_trace(
            go.Bar(
                name=f'{metric} - Current',
                x=[metric],
                y=[values[0]],
                marker_color=BUSINESS_COLORS['primary'],
                showlegend=(i==0)
            ),
            row=1, col=2
        )
        
        fig_health.add_trace(
            go.Bar(
                name=f'{metric} - Previous',
                x=[metric],
                y=[values[1]],
                marker_color=BUSINESS_COLORS['neutral'],
                showlegend=(i==0)
            ),
            row=1, col=2
        )

# Revenue vs satisfaction scatter (if available)
if 'avg_review_score' in executive_summary:
    fig_health.add_trace(
        go.Scatter(
            x=[executive_summary['avg_review_score']],
            y=[executive_summary['total_revenue']],
            mode='markers',
            name='Current Period',
            marker=dict(size=20, color=BUSINESS_COLORS['primary'])
        ),
        row=2, col=1
    )

# Overall health gauge
fig_health.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=health_metrics['overall_health_score'],
        title={'text': "Overall Health Score"},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': BUSINESS_COLORS['primary']},
            'steps': [
                {'range': [0, 50], 'color': BUSINESS_COLORS['danger']},
                {'range': [50, 80], 'color': BUSINESS_COLORS['warning']},
                {'range': [80, 100], 'color': BUSINESS_COLORS['success']}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 90
            }
        }
    ),
    row=2, col=2
)

fig_health.update_layout(
    height=800,
    title_text=f"Business Health Dashboard - {period_label}",
    title_x=0.5
)

# Update y-axis labels
fig_health.update_yaxes(title_text="Health Score (0-100)", row=1, col=1)
fig_health.update_yaxes(title_text="Revenue ($)", row=2, col=1)

# Update x-axis labels
fig_health.update_xaxes(title_text="Customer Satisfaction (1-5)", row=2, col=1)

fig_health.show()

## 9. Key Insights & Recommendations

### Summary of Key Findings

Based on the comprehensive analysis of the e-commerce data, here are the primary insights:

#### Revenue Performance
- **Revenue Trend**: Analysis reveals the overall revenue performance and growth trajectory
- **Order Volume**: Changes in order volume indicate market demand patterns
- **Average Order Value**: AOV trends show customer spending behavior changes

#### Product Portfolio
- **Category Performance**: Top-performing product categories drive majority of revenue
- **Market Share**: Revenue concentration across categories shows business focus areas
- **Pricing Strategy**: Average prices vary significantly across product categories

#### Geographic Distribution
- **Market Concentration**: Revenue is concentrated in key geographic markets
- **Customer Density**: Some states show high customer acquisition potential
- **Market Efficiency**: Revenue per customer varies by geographic region

#### Customer Experience
- **Satisfaction Levels**: Overall customer satisfaction based on review scores
- **Delivery Performance**: Average delivery times and their impact on satisfaction
- **Service Quality**: Correlation between operational metrics and customer feedback

### Strategic Recommendations

1. **Revenue Growth**
   - Focus on high-performing product categories for expansion
   - Implement targeted pricing strategies based on category performance
   - Develop retention programs to increase customer lifetime value

2. **Product Strategy**
   - Invest in top-performing categories with strong market demand
   - Evaluate underperforming categories for potential optimization or discontinuation
   - Consider cross-selling opportunities between complementary categories

3. **Geographic Expansion**
   - Prioritize marketing investments in high-revenue states
   - Develop market entry strategies for underrepresented regions
   - Optimize logistics and fulfillment for key geographic markets

4. **Customer Experience Enhancement**
   - Focus on reducing delivery times to improve satisfaction scores
   - Implement customer feedback loops to address service issues
   - Develop loyalty programs to increase customer retention

5. **Operational Improvements**
   - Optimize fulfillment processes to reduce delivery times
   - Implement predictive analytics for demand forecasting
   - Enhance supply chain efficiency in key markets

### Next Steps

1. **Regular Monitoring**: Implement this analysis as a recurring monthly/quarterly review
2. **A/B Testing**: Test recommendations through controlled experiments
3. **Advanced Analytics**: Develop predictive models for customer behavior and demand forecasting
4. **Real-time Dashboards**: Create operational dashboards for day-to-day monitoring
5. **Competitive Analysis**: Include competitive benchmarking in future analyses

---

### Configuration Notes

This analysis framework is designed to be reusable and configurable:

- **Time Periods**: Easily modify analysis and comparison periods in the configuration section
- **Geographic Scope**: Adjust geographic analysis based on business expansion plans
- **Product Focus**: Modify category analysis depth based on business priorities
- **Metrics**: Add or remove KPIs based on evolving business needs

For questions about this analysis or to request modifications, please refer to the README documentation.